In [1]:
import pandas as pd

In [2]:
breaches = pd.read_csv("Balloon Race Data Breaches - LATEST - breaches.csv")

In [3]:
breaches['organisation'] = breaches['organisation'].str.lower() 

In [4]:
stocks = pd.read_csv("World-Stock-Prices-Dataset.csv")

In [5]:
stocks['Brand_Name'] = stocks['Brand_Name'].str.lower() 

In [6]:
stocks = stocks.rename(columns={'Brand_Name': 'organisation'})

In [7]:
stocks = stocks.rename(columns={'Date': 'StockDate'})

In [8]:
breaches = breaches.rename(columns={'date': 'BreachDate'}) 

In [9]:
breaches.dtypes

organisation          object
alternative name      object
records lost          object
year                  object
BreachDate            object
story                 object
sector                object
method                object
interesting story     object
data sensitivity      object
displayed records     object
Unnamed: 11          float64
source name           object
1st source link       object
2nd source link       object
ID                   float64
dtype: object

In [10]:
breaches = breaches.drop(columns=["alternative name","story","interesting story","Unnamed: 11", "source name", "1st source link", "2nd source link", "ID"])

In [11]:
stocks = stocks.drop(columns=["Open","High","Low","Ticker","Dividends","Stock Splits","Capital Gains"])

In [12]:
stocks.dtypes

StockDate        object
Close           float64
Volume          float64
organisation     object
Industry_Tag     object
Country          object
dtype: object

In [13]:
merged_data_stocks = pd.merge(stocks, breaches, on='organisation', how='left')

In [14]:
merged_data_stocks = merged_data_stocks.dropna(subset=['organisation','records lost']) 

In [15]:
distinct_values = merged_data_stocks['organisation'].unique()

In [16]:
print(distinct_values)

['peloton' 'amazon' 'apple' 'target' 'spotify' 'starbucks' 'nvidia'
 'adobe' 'toyota' 'uber' 'microsoft' 'ubisoft' 'nintendo' 'twitter']


In [17]:
print(merged_data_stocks)

                        StockDate       Close       Volume organisation  \
0       2025-07-03 00:00:00-04:00    6.640000    4209664.0      peloton   
3       2025-07-03 00:00:00-04:00  223.410004   29295154.0       amazon   
4       2025-07-03 00:00:00-04:00  213.550003   34697317.0        apple   
6       2025-07-03 00:00:00-04:00  104.059998    3535290.0       target   
8       2025-07-03 00:00:00-04:00  725.049988    1071771.0      spotify   
...                           ...         ...          ...          ...   
318171  2000-01-03 00:00:00-05:00    2.437765   24232000.0    starbucks   
318178  2000-01-03 00:00:00-05:00    4.468750  322352000.0       amazon   
318181  2000-01-03 00:00:00-05:00    3.990652     134500.0     nintendo   
318182  2000-01-03 00:00:00-05:00    3.990652     134500.0     nintendo   
318188  2000-01-03 00:00:00-05:00   72.034958      21200.0       toyota   

           Industry_Tag Country records lost year    BreachDate     sector  \
0               fitne